In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Point Wise**

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Data
data = pd.DataFrame({
    "query_id": [1, 1, 1, 2, 2, 2],
    "f1": [0.1, 0.4, 0.3, 0.9, 0.2, 0.5],
    "f2": [0.7, 0.1, 0.4, 0.2, 0.9, 0.6],
    "relevance": [3, 2, 1, 2, 3, 0]  # nhãn: 0-3
})

X = data[["f1", "f2"]]
y = data["relevance"]

# Học mô hình hồi quy
model = LinearRegression()
model.fit(X, y)

# Dự đoán relevance
data["pred_score"] = model.predict(X)
print("Kết quả dự đoán")
print(data[["query_id", "f1", "f2", "relevance", "pred_score"]])

Kết quả dự đoán
   query_id   f1   f2  relevance  pred_score
0         1  0.1  0.7          3    2.287212
1         1  0.4  0.1          2    1.728372
2         1  0.3  0.4          1    1.942033
3         2  0.9  0.2          2    1.098165
4         2  0.2  0.9          3    2.210457
5         2  0.5  0.6          0    1.733761


**Pairwise**

* Xem bài toán như so sánh cặp document trong cùng querry , mục đích là để so sánh xem document nào sẽ rank cao hơn

In [8]:
from sklearn.linear_model import LogisticRegression
from itertools import combinations

# Data
pairs = []
labels = []

for qid in data["query_id"].unique():
    docs = data[data["query_id"] == qid]
    for (i, di), (j, dj) in combinations(docs.iterrows(), 2):
        diff = di[["f1", "f2"]] - dj[["f1", "f2"]]
        label = 1 if di["relevance"] > dj["relevance"] else 0
        pairs.append(diff.values)
        labels.append(label)

X_pairs = np.vstack(pairs)
y_pairs = np.array(labels)

# Train logistic regression 
pair_model = LogisticRegression()
pair_model.fit(X_pairs, y_pairs)

print("Pairwise model accuracy:", pair_model.score(X_pairs, y_pairs))

Pairwise model accuracy: 0.8333333333333334


In [9]:
import xgboost as xgb

# Data
X = data[["f1", "f2"]].values
y = data["relevance"].values
group = data.groupby("query_id").size().to_list() 

dtrain = xgb.DMatrix(X, label=y)
dtrain.set_group(group)

params = {
    "objective": "rank:ndcg",   # Listwise loss
    "eval_metric": "ndcg",
    "eta": 0.1,
    "max_depth": 3
}

rank_model = xgb.train(params, dtrain, num_boost_round=20)

# Dự đoán
data["pred_score"] = rank_model.predict(dtrain)
print(data[["query_id", "relevance", "pred_score"]])

   query_id  relevance    pred_score
0         1          3  1.310156e-08
1         1          2  1.310156e-08
2         1          1  1.310156e-08
3         2          2  1.310156e-08
4         2          3  1.310156e-08
5         2          0  1.310156e-08
